In [416]:
import pandas as pd
import numpy as np
import plotly.express as px 


In [417]:
discount_rate = 0.0541 #nominal
inflation = 0.003
monthly_solar_radiation = [5.9*31, 6.4*28, 7*31, 7*30, 6.8*31, 6.9*30, 7.1*31, 7.2*31, 6.7*30, 6.1*31, 5.6*30, 5.3*31]
degradation_rate = 0.00469
electricity_tariff = 0.5444
electricity_escalation_rate = 0.00208
feed_in_tariff = electricity_tariff*0.50
kw10 = 9.787
ic10 = 19667
kw100 = 99.920
ic100 = 180833
kw50 = 49.813
ic50 = 91571


In [418]:
eg = list(i * 10 for i in monthly_solar_radiation)
eg.reverse()
print((eg))

[1642.9999999999998, 1680.0, 1891.0, 2010.0, 2232.0, 2201.0, 2070.0, 2108.0, 2100, 2170, 1792.0000000000002, 1829.0]


In [419]:
def NPV_formula(eer,inf, kw, ic, d, msr, dr, et, fit, eu, delay): #delay=1 if acquiring a permit takes 2 years, 0 if permit is issued immediately 
    NPV =[]
    for l in eu:
        npv = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                yearly_value += monthly_value
            npv+=(yearly_value/(1+d)**(i+1))
        NPV.append(round(npv))
    return NPV        

In [420]:
def SNPV_formula(eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    NPV = -ic
    for i in range(25):
        yearly_value = 0
        monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
        for k in monthly_electricity_generated:
            monthly_value = kw*-15*(1+inf)**(i+1)
            if k > eu:
                monthly_value += eu * et*(1+eer)**(i+1)
                if delay != 2 and i > 1:
                    monthly_value += (k-eu)*fit*(1+eer)**(i+1)
            else:
                monthly_value += k*et*(1+eer)**(i+1)
            yearly_value += monthly_value
        NPV+=(yearly_value/(1+d)**(i+1))
    return NPV        

In [421]:
def SPP_formula(eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    SPP = []
    for l in eu:
        NPV = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                yearly_value += monthly_value
            NPV+=(yearly_value/(1+d)**(i+1))
            if NPV > 0:
                spp = i+1
                break
        if NPV > 0:
            monthly_electricity_generated.reverse()
            month_counter = 1
            for k in monthly_electricity_generated:
                month_counter -= 0.083
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                NPV -= monthly_value/(1+d)**(i+1)
                if NPV < 0:
                    spp += round(month_counter,2)
                    monthly_electricity_generated.reverse()
                    break
            SPP.append(spp)
        else:
            SPP.append(0)
    return SPP

In [422]:
def IRR_formula(eer, inf, kw, ic, msr, dr, et, fit, eu, delay):
    IRR = []
    for i in eu:
        lower_bound = -0.5
        upper_bound = 1
        tolerance = 1e-4
        max_iterations = 100
        iteration = 0

        while iteration < max_iterations:
            mid_rate = (lower_bound + upper_bound) / 2
            if mid_rate == 0:
                mid_rate += 1e-4
            npv = SNPV_formula(eer, inf, kw, ic, mid_rate, msr, dr, et, fit, i, delay)
            if abs(npv) < tolerance: 
                IRR.append(round(mid_rate*100, 3))
                break
            elif npv > 0:
                lower_bound = mid_rate  
            else:
                upper_bound = mid_rate 
            iteration += 1
        if iteration >= max_iterations:
            IRR.append(-100)  #IRR is too small to calculate
    return IRR

In [436]:
def LCOE_formula(inf, kw, ic, d, msr, dr, et, fit, eu):
    NPV_costs = ic
    NPV_energy = 0
    for i in range(25):
        NPV_costs += 12*kw*15*(1+inf)**(1+i)/(1+d)**(i+1)
        NPV_energy += (sum(msr) * kw * (1 - dr*i)) / (1+d)**(i+1)
    return NPV_costs/NPV_energy
    

In [485]:
def ROI_formula (eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    ROI = []
    for j in eu:
        NPV_return = SNPV_formula(eer, inf, kw, ic, d, msr, dr, et, fit, j, delay)
        NPV_costs = ic
        for i in range(25):
            NPV_costs += 12*kw*15*(1+inf)**(1+i)/(1+d)**(i+1)
        ROI.append((NPV_return/NPV_costs)*100)
    return ROI

In [487]:
def ROI_graph(x_range):
    x = np.array(x_range)
    y10 = (ROI_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y10d = (ROI_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (ROI_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    y100d = (ROI_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (ROI_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})

    y50d = (ROI_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'ROI of a different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'ROI (%)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='green', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='red', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='green', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='red', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='green', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='red',dash='dot'))

    fig.show()

In [424]:
def IRR_graph(x_range):
    x = np.array(x_range)
    y10 = (IRR_formula(electricity_escalation_rate, inflation, kw10, ic10, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})
    y10d = (IRR_formula(electricity_escalation_rate, inflation, kw10, ic10, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (IRR_formula(electricity_escalation_rate, inflation, kw100, ic100, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})
    y100d = (IRR_formula(electricity_escalation_rate, inflation, kw100, ic100, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (IRR_formula(electricity_escalation_rate, inflation, kw50, ic50, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})
    y50d = (IRR_formula(electricity_escalation_rate, inflation, kw50, ic50, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'IRR of different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'IRR in %'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='green', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='red', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='green', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='red', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='green', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='red', dash='dot'))    

    fig.update_yaxes(range=[-25, 75])
    
    fig.show()

In [425]:
def SPP_graph(x_range):
    x = np.array(x_range)
    y10 = (SPP_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})
    y10d = (SPP_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (SPP_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})
    y100d = (SPP_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})
    df100d = df100d[df100d['y'] > 0]

    y50 = (SPP_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})
    y50d = (SPP_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})
    df50d = df50d[df50d['y'] > 0]

    fig = px.line(df10, x='x', y='y', title=f'SPP of different systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'SPP (years)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='green', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='red', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='green', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='red', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='green', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='red', dash='dot'))

    fig.update_yaxes(range=[1, 30])

    fig.show()

In [433]:
def LCOE_graph(x_range):
    x = np.array(x_range)
    y10 = (LCOE_formula(inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y100 = (LCOE_formula(inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    y50 = (LCOE_formula(inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df50 = pd.DataFrame({'x': x, 'y': y50})

    fig = px.line(df10, x='x', y='y', title=f'LCOE of different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'LCOE (kWh/AWG)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(dash='dot'))

    fig.update_yaxes(range=[0.13, 0.15])

    fig.show()

In [427]:
def NPV_graph(x_range):
    x = np.array(x_range)
    y10 = (NPV_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y10d = (NPV_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (NPV_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    y100d = (NPV_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (NPV_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})

    y50d = (NPV_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'NPV of a different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'NPV (AWG)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='green', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='red', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='green', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='red', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='green', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='red',dash='dot'))

    fig.show()

In [488]:
ROI_graph(x_range=np.linspace(1000, 30000, 100))

In [428]:
IRR_graph(x_range=np.linspace(1000, 30000, 100))

In [429]:
SPP_graph(x_range=np.linspace(1000, 25000, 1000))

In [475]:
LCOE_graph(x_range=np.linspace(1000, 100000, 1000))

In [ ]:
NPV_graph(x_range=np.linspace(1000, 30000, 1000))
          

In [453]:
def optimal_calculator(eu, kw, ic):
    optimal = []
    for i in eu:
        for j in range(len(kw)):
            highest_npv = 0
            npv_current  = SNPV_formula(electricity_escalation_rate, inflation, kw[j], ic[j], discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, i, 2)    
            if highest_npv < npv_current:
                highest_npv = npv_current
                optimal_kw = kw[j]
        optimal.append(optimal_kw)
    return optimal
         

In [498]:
def ic_calculator(kw):
    return kw*1788.0+2166.95

monthly_electricity_consumption = np.linspace(1000, 100000, 100)
kw_options = np.linspace(10, 1100, 220)
according_initial_cost = []
for kw in kw_options:
    according_initial_cost.append(ic_calculator(kw))


In [499]:
def optimal_graph(eu, kw, ic):
    x = eu
    y = optimal_calculator(eu, kw, ic)
    df = pd.DataFrame({'x': x, 'y': y})

    fig = px.line(df, x='x', y='y', title='Optimal amount of kWp for maximum NPV considering permit delay', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'Amount of kWp installed'})

    fig.add_scatter(x=df['x'], y=df['y'], mode='lines', name='Most valuable kWp installation given electricity usage', line=dict(color='blue', dash='solid'))

    fig.add_trace(go.Scatter(
        x=[min(x), max(x)], 
        y=[100, 100],        
        mode='lines',
        name='Maximum amount of kWp allowed by the government',  
        line=dict(color='red', width=2)
        ))
    fig.update_xaxes(range=[0, 50000])
    fig.update_yaxes(range=[0, 1000])

    fig.show()

In [500]:
optimal_graph(monthly_electricity_consumption, kw_options, according_initial_cost)